<a href="https://colab.research.google.com/github/LeonVillanueva/CoLab/blob/master/Google_CoLab_Dip_Passive_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import datetime as dt

In [0]:
df = pd.read_csv ('VTSMX.csv')

In [0]:
df['Month'] = pd.DatetimeIndex (df['Date']).month

In [12]:
df['Close'].shi

,Date,Open,High,Low,Close,Adj Close,Volume,Month
0,1992-04-27,10.03,10.03,10.03,10.03,5.913749,0,4
1,1992-04-28,10.00,10.00,10.00,10.00,5.896062,0,4
2,1992-04-29,10.00,10.00,10.00,10.00,5.896062,0,4
3,1992-04-30,10.18,10.18,10.18,10.18,6.002193,0,4
4,1992-05-01,10.18,10.18,10.18,10.18,6.002193,0,5
